### Enumerations

We'll need the `enum` module:

In [1]:
import enum

The base class for enums is `Enum`. To create an enumeration we need to **subclass** it:

In [2]:
class Color(enum.Enum):
    red = 1
    green = 2
    blue = 3

Associated values can be anything, not just integer values:

In [3]:
class Status(enum.Enum):
    PENDING = 'pending'
    RUNNING = 'running'
    COMPLETED = 'completed'    

In [4]:
class UnitVector(enum.Enum):
    V1D = (1, )
    V2D = (1, 1)
    V3D = (1, 1, 1)

Each member of an enumeration has a type of the enumeration class itself:

In [5]:
Status.PENDING

<Status.PENDING: 'pending'>

In [6]:
type(Status.PENDING)

<enum 'Status'>

In [7]:
isinstance(Status.PENDING, Status)

True

Each member (instance of the enumeration) has properties, just like any object:

In [8]:
Status.PENDING.name, Status.PENDING.value

('PENDING', 'pending')

Although `==` is supported, member equality is generally tested using identity, `is`. It is also faster than using `==`:

In [9]:
Status.PENDING is Status.PENDING

True

In [10]:
Status.PENDING == Status.PENDING

True

Note that although `==` (and `!=`) is supported, rich comparison operators are not (it would not make sense, except maybe if the values are values such as integers - we'll come back to that):

In [11]:
class Constants(enum.Enum):
    ONE = 1
    TWO = 2
    THREE = 3

In [12]:
try:
    Constants.ONE > Constants.TWO
except TypeError as ex:
    print(ex)

'>' not supported between instances of 'Constants' and 'Constants'


Membership can be tested using `in`:

In [13]:
Status.PENDING in Status

True

Note that the names (strings) and associated values are not themselves members of the enumeration - remember that enumeration members are instances of the enumeration class:

In [14]:
Status.PENDING.name, Status.PENDING.value

('PENDING', 'pending')

In [15]:
'PENDING' in Status, 'pending' in Status

/var/folders/gk/qmm3q5ld1b74kcdmh5pnm1xw0000gn/T/ipykernel_77971/1927625244.py:1: DeprecationWarning: in 3.12 __contains__ will no longer raise TypeError, but will return True if
obj is a member or a member's value
  'PENDING' in Status, 'pending' in Status


TypeError: unsupported operand type(s) for 'in': 'str' and 'EnumMeta'

Enums are callables, and we can look up a member by **value** by calling the enumeration:

In [ ]:
Status('pending'), UnitVector((1,1))

(<Status.PENDING: 'pending'>, <UnitVector.V2D: (1, 1)>)

But if we try to lookup a member with a non-existent value, we get a `ValueError` exception:

In [16]:
try:
    Status('invalid')
except ValueError as ex:
    print(ex)

'invalid' is not a valid Status


Recall that a class that implements the `__getitem__` method supports the [] operation:

In [17]:
class Person:
    def __getitem__(self, val):
        return f'__getitem__({val}) called...'

In [18]:
p = Person()
p['some value']

'__getitem__(some value) called...'

Enumerations implement this `__getitem__` method:

In [51]:
hasattr(Status, '__getitem__')

True

So we can look up a member by it's name (think of it as a key):

In [52]:
Status['PENDING']

<Status.PENDING: 'pending'>

But the enumeration members, although instances of the enumeration, are also class attributes of the enumeration, so we can also use `getattr` like we would with any standard class attribute:

In [21]:
getattr(Status, 'PENDING')

<Status.PENDING: 'pending'>

Enumeration members are always hashable, even if their associated values are not (makes sense, since member names are basically strings):

In [53]:
class Person:
    __hash__ = None

In [54]:
p = Person()
try:
    hash(p)
except TypeError as ex:
    print(ex)

unhashable type: 'Person'


So, although `Person` objects are not hashable:

In [55]:
class Family(enum.Enum):
    person_1 = Person()
    person_2 = Person()

In [56]:
Family.person_1

<Family.person_1: <__main__.Person object at 0x110831600>>

We can still use members as keys in a dictionary:

In [57]:
{
    Family.person_1: 'person 1',
    Family.person_2: 'person 2'
}

{<Family.person_1: <__main__.Person object at 0x110831600>>: 'person 1',
 <Family.person_2: <__main__.Person object at 0x110830400>>: 'person 2'}

Enumerations are iterables:

In [58]:
hasattr(Status, '__iter__')

True

So we can iterate over the members:

In [59]:
for member in Status:
    print(repr(member))

<Status.PENDING: 'pending'>
<Status.RUNNING: 'running'>
<Status.COMPLETED: 'completed'>


Note that iteration order is the order in which the members are declared in the enumeration, and has nothing to do with the associated values:

In [60]:
class Numbers1(enum.Enum):
    ONE = 1
    TWO = 2
    THREE = 3
    
class Numbers2(enum.Enum):
    THREE = 3
    TWO = 2
    ONE = 1

In [61]:
list(Numbers1)

[<Numbers1.ONE: 1>, <Numbers1.TWO: 2>, <Numbers1.THREE: 3>]

In [62]:
list(Numbers2)

[<Numbers2.THREE: 3>, <Numbers2.TWO: 2>, <Numbers2.ONE: 1>]

Lastly, enumerations are immutable: we cannot add/remove elements from the enumeration, **and** we canniot modify the associated values:

In [63]:
try:
    Status.PENDING.value = 10
except AttributeError as ex:
    print(ex)

can't set attribute


In [64]:
try:
    Status['NEW'] = 100
except TypeError as ex:
    print(ex)

'EnumMeta' object does not support item assignment


We'll come back to this later, but we cannot extend an enumeration once it has members defined:

In [65]:
class EnumBase(enum.Enum):
    pass

In [66]:
class EnumExt(EnumBase):
    ONE = 1
    TWO = 2

In [67]:
EnumExt.ONE

<EnumExt.ONE: 1>

But this would not work:

In [68]:
class EnumBase(enum.Enum):
    ONE = 1

In [69]:
try:
    class EnumExt(EnumBase):
        TWO = 2
except TypeError as ex:
    print(ex)

EnumExt: cannot extend enumeration 'EnumBase'


##### Example

So the basics of enumerations are quite straightforward. You might be wondering though why we have two ways of referencing members by name:

In [70]:
Status.PENDING, Status['PENDING']

(<Status.PENDING: 'pending'>, <Status.PENDING: 'pending'>)

This is because sometimes we might get a string from some input, and need to match it up with a member in the enumeration.

For example it might be a status that comes back from an API call in a JSON payload:

In [71]:
payload = """
{
  "name": "Alex",
  "status": "PENDING"
}
"""

In [72]:
import json

data = json.loads(payload)

In [73]:
data['status']

'PENDING'

And now we can look up the status in the enumeration, but we have to use the `__getitem__` method:

In [74]:
Status[data['status']]

<Status.PENDING: 'pending'>

##### Example 2

A natural question given the last example might be: how do we determine if some string corresponds to a member name in our enumeration?

We have three basic ways of doing this.

First we could simply lookup the value by name, and trap the `KeyError` exception:

In [75]:
def is_member(en, name):
    try:
        en[name]
    except KeyError:
        return False
    return True

In [45]:
is_member(Status, 'PENDING')

True

In [76]:
is_member(Status, 'pending')

False

We could also just use the `getattr` function:

In [77]:
getattr(Status, 'PENDING', None), getattr(Status, 'OK', None)

(<Status.PENDING: 'pending'>, None)

But we could also just use the `__members__` property:

In [78]:
Status.__members__

mappingproxy({'PENDING': <Status.PENDING: 'pending'>,
              'RUNNING': <Status.RUNNING: 'running'>,
              'COMPLETED': <Status.COMPLETED: 'completed'>})

As you can see we get a `mappingproxy` object back, so we can use membership in that object (that defaults to using the keys), or the `keys()` view if we want to be more explicit:

In [79]:
'PENDING' in Status.__members__

True

In [80]:
'PENDING' in Status.__members__.keys()

True